In [1]:
import numpy as np
import torch 
import os, copy
from collections import OrderedDict

from utils.sample_parameters import ParamSamples
from utils.train_n_test import TrainTest
from utils.custom_loss import opportunity_loss

from models.CNN_1D import CNN_1D

In [2]:
balance_params = {
    'START': 1e7, 
    'REWARD': 10, 
    'FINE': 100, 
    'NORM_HYPERPARAMS': (0,87000),
    'NORM': 'minmax',
    'WARM': 1
}

params_dict = {
    # HYPERMARAMETER RANGES
    'BATCH_SIZE': ([64, 128, 256, 512], None),
    'EPOCHS': ([20, 40, 50], None),
    'OPTIMIZER': ([torch.optim.RMSprop, torch.optim.Adam], None),
    'LEARNING_RATE': ([1e-5, 5e-5, 1e-4, 5e-4, 1e-3], None),
    'LOSS': ([revenue_loss()], None),
    'EARLY_STOPPING': ([True], None),
    'PATIENCE': (list(range(0,3)), None),
    'MIN_DELTA': ([1e-6, 5e-6, 1e-5, 5e-5, 1e-4, 5e-4], None), 

    # NETWORK PARAMETER RANGES
    'HIDDEN_DEPTH': (np.arange(2, 5), None),
    'HIDDEN_SIZES': ([64, 128, 256, 512], 
                       {'max_depth': 'HIDDEN_DEPTH', # sample a list of maximum HIDDEN_DEPTH length 
                        'consecutive': 1}), # list must be increasing 
    'KERNEL_SIZES': ([3,5,7,9], 
                        {'max_depth': 'HIDDEN_DEPTH', 
                        'monotonic': -1}),  
    'MAXPOOL': ([2], None),
    'FC_DEPTH':  (np.arange(1,3), None),
    'FC_SIZES': ([256, 512], 
                    {'max_depth': 'FC_DEPTH',
                    'consecutive': -1}),
    'DROPOUT': (np.arange(8)/10, None),
    'BALANCE': ([balance_params], None)
}

In [34]:
fixed_params = [{'BATCH_SIZE': 256,
  'EPOCHS': 20,
  'OPTIMIZER': torch.optim.RMSprop,
  'LEARNING_RATE': 0.0001,
  'LOSS': torch.nn.MSELoss(), 
  'EARLY_STOPPING': True,
  'PATIENCE': 1,
  'MIN_DELTA': 1e-06,
  'HIDDEN_DEPTH': 4,
  'HIDDEN_SIZES': np.array([64,128, 256, 512]),
  'KERNEL_SIZES': np.array([9, 7, 5, 3]),
  'MAXPOOL': 2,
  'FC_DEPTH': 2,
  'FC_SIZES': np.array([512]),
  'DROPOUT': 0.2}]

In [4]:
N = 5
FIX = True
PKL_NAMES = [i for i in os.listdir('data/tensors') if 'expt0' in i ]
PKL_NAMES = [i for i in PKL_NAMES if 'persistence' not in i ]

In [ ]:
for pkl in PKL_NAMES:  
    PKL_NAME = pkl[:-4]
    print(PKL_NAME)
    train, val, test = torch.load('data/tensors/'+ PKL_NAME + '.pkl')
    persistence = torch.load('data/tensors/'+ PKL_NAME + '_persistence.pkl')
    
    torch.nn.MSELoss()(persistence, test[1])
    
    if FIX: 
        param_samples = [copy.deepcopy(fixed_params[0]) for _ in range(N)]        
    else: 
        params = ParamSamples(OrderedDict(params_dict))
        param_samples = params.SampleAll(N)

    stats = []
    for i in np.arange(N): 
        print(param_samples[i])
        run = TrainTest(CNN_1D, (train, val, test), param_samples[i], class_size=1)
        run.train()
        run.test()
        stats.append(run.stats)
    
        for i in np.arange(N):
            for key,val in stats[i].items():
                param_samples[i][key] = val

        torch.save(param_samples, 'results/CNN1D_experiment_N'+str(N)+'_'+PKL_NAME+'.pkl')

In [ ]:
# Can't find a way to make this a function (plot not showing), so putting this in every notebook 
# from matplotlib.animation import FuncAnimation, PillowWriter 
# %matplotlib notebook

# interval = 50
# windowsize = 100
# test_ = test[1]
# pred = stats[0]['predictions']

# fig  = plt.figure()
# ax   = plt.axes(xlim=(0, windowsize), ylim=(-2, 2))
# ln1, = plt.plot([],[])
# ln2, = plt.plot([],[])

# def update_animation(f):
#     r = np.arange(f, f + windowsize)
#     lim = ax.set_xlim(f, f+ windowsize)
#     ln1.set_data(r, test_[r])
#     ln2.set_data(r, pred[r])

# frames = np.arange(0, len(pred)-windowsize, interval)
# ani = FuncAnimation(fig, update_animation, frames=frames) 
# plt.show()
# #ani.save('results/gif/'+PKL_NAME+'.gif', writer='imagemagick', fps=50)